In [1]:
# Basic Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# PyTorch Libraries
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim


from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image
import random

In [2]:
# ✅ New Dataset Path
base_path = 'Images and labels/All_Breeds_Dataset'
labels_df = pd.read_csv('Images and labels/final_labels.csv')

labels_df['Path'] = labels_df['Path'].apply(lambda x: os.path.join("Images and labels", x.replace('\\', '/')))

# ✅ Dataset Info
print("Total Images:", len(labels_df))
print("Total Breeds:", labels_df['Breed Name'].nunique())
labels_df.head()

Total Images: 1336
Total Breeds: 23


,Image ID,Path,Breed Name,Coat Length,Personality
0,Abyssinian_01,Images and labels\All_Breeds_Dataset/Abyssinia...,Abyssinian,ShortHair,"Busy, active, agenda-driven and affectionate"
1,Abyssinian_02,Images and labels\All_Breeds_Dataset/Abyssinia...,Abyssinian,ShortHair,"Busy, active, agenda-driven and affectionate"
2,Abyssinian_03,Images and labels\All_Breeds_Dataset/Abyssinia...,Abyssinian,ShortHair,"Busy, active, agenda-driven and affectionate"
3,Abyssinian_04,Images and labels\All_Breeds_Dataset/Abyssinia...,Abyssinian,ShortHair,"Busy, active, agenda-driven and affectionate"
4,Abyssinian_05,Images and labels\All_Breeds_Dataset/Abyssinia...,Abyssinian,ShortHair,"Busy, active, agenda-driven and affectionate"


In [3]:
# ✅ Custom Dataset Class (no changes needed)
from torch.utils.data import Dataset
from PIL import Image
import os

class CatBreedDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.copy()
        self.transform = transform
        self.classes = sorted(df['Breed Name'].unique())
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['Path']  # Already correct relative path
        image = Image.open(image_path).convert('RGB')
        label = self.class_to_idx[row['Breed Name']]
        if self.transform:
            image = self.transform(image)
        return image, label


In [4]:
# ✅ Transforms
from torchvision import transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.75, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.4, 0.4, 0.4),
        transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
        transforms.GaussianBlur(kernel_size=3),
        AutoAugment(policy=AutoAugmentPolicy.IMAGENET),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
        transforms.RandomErasing(p=0.2, scale=(0.02, 0.1), ratio=(0.3, 3.3), value='random')
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}

In [5]:
# ✅ Dataset Load & Split
from torch.utils.data import DataLoader, random_split

full_dataset = CatBreedDataset(labels_df, transform=data_transforms['train'])
train_size = int(0.6 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_data, val_data, test_data = random_split(full_dataset, [train_size, val_size, test_size])

batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [6]:
# ✅ EfficientNetB0 (Strategy S4)
from torchvision import models
import torch.nn as nn

def prepare_efficientnet_s4(num_classes):
    model = models.efficientnet_b0(pretrained=True)
    for name, param in model.features.named_parameters():
        if not name.startswith('0.'):
            param.requires_grad = True
        else:
            param.requires_grad = False
    for param in model.classifier.parameters():
        param.requires_grad = True
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

In [7]:
# ✅ Training Loop
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, strategy_name='', epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    best_acc = 0
    best_epoch = 0

    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()

        acc = 100 * correct / total
        print(f"[{strategy_name}] Epoch {epoch+1}: Val Accuracy = {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            best_epoch = epoch + 1
            torch.save(model, "efficientnet_s4_best.pt")

        if scheduler:
            scheduler.step(acc)

    print(f"[{strategy_name}] Best Val Accuracy = {best_acc:.2f}% at Epoch {best_epoch}")
    return model

In [8]:
# ✅ Start Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = labels_df['Breed Name'].nunique()

model = prepare_efficientnet_s4(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, patience=3, factor=0.3)

model = train_model(
    model, train_loader, val_loader,
    criterion, optimizer, scheduler,
    strategy_name='S4', epochs=10
)

[S4] Epoch 1: Val Accuracy = 54.89%
[S4] Epoch 2: Val Accuracy = 57.89%
[S4] Epoch 3: Val Accuracy = 48.87%
[S4] Epoch 4: Val Accuracy = 62.41%
[S4] Epoch 5: Val Accuracy = 73.68%
[S4] Epoch 6: Val Accuracy = 75.19%
[S4] Epoch 7: Val Accuracy = 63.91%
[S4] Epoch 8: Val Accuracy = 71.43%
[S4] Epoch 9: Val Accuracy = 75.19%
[S4] Epoch 10: Val Accuracy = 81.20%
[S4] Best Val Accuracy = 81.20% at Epoch 10
